In [1]:
!pip install datasets gensim spacy
# !python -m spacy download ro_core_news_sm

  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached typer_slim-0.21.1-py3-none-any.whl.metadata (16 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
  Using cached cloudpathlib-0.23.0-py3-none-any.whl.metadata (16 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 39.1 MB/s eta 0:00:0031m39.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 42.9 MB/s eta 0:00:00
Using cached catalogue-2.0.10-py3-none-any.whl (17 kB)
Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
Using cached spacy_loggers-1.0.5-py3-none-any.whl (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.2/653.2 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 741.1/741.1 kB 39.2 MB/s eta 0:00:00
Using cach

In [7]:
# from datasets import load_dataset
# import spacy
# from tqdm.auto import tqdm

# dataset = load_dataset("andyP/ro-paraphrase-bible")
# nlp = spacy.load("ro_core_news_sm", disable=['ner', 'parser'])

# text_columns = [col for col in dataset['train'].column_names if dataset['train'].features[col].dtype == 'string']
# col1, col2 = text_columns[0], text_columns[1]

# list1 = [s for s in tqdm(dataset['train'][col1], desc="Loading Column 1") if s is not None]
# list2 = [s for s in tqdm(dataset['train'][col2], desc="Loading Column 2") if s is not None]

# all_sentences = set(list1 + list2)
# original_sentences_list = list(all_sentences)

def preprocess(text):
    doc = nlp(str(text))
    return [token.lemma_.lower() for token in doc if not token.is_punct and not token.is_space]

# processed_corpus = [preprocess(sent) for sent in tqdm(original_sentences_list, desc="Tokenizing Romanian Text")]

# print(f"Finished processing {len(processed_corpus)} sentences.")

In [10]:
from datasets import load_dataset
import spacy
from tqdm.auto import tqdm

dataset = load_dataset("andyP/ro-paraphrase-bible")
nlp = spacy.load("ro_core_news_sm", disable=['ner', 'parser'])

text_columns = [col for col in dataset['train'].column_names if dataset['train'].features[col].dtype == 'string']
col1, col2 = text_columns[0], text_columns[1]

list1 = [s for s in tqdm(dataset['train'][col1], desc="Loading Column 1") if s is not None]
list2 = [s for s in tqdm(dataset['train'][col2], desc="Loading Column 2") if s is not None]

all_sentences = set(list1 + list2)
original_sentences_list = list(all_sentences)

processed_corpus = []

for doc in tqdm(nlp.pipe(original_sentences_list, batch_size=3000, n_process=-1),
                total=len(original_sentences_list),
                desc="Parallel Tokenizing"):
    tokens = [token.lemma_.lower() for token in doc if not token.is_punct and not token.is_space]
    processed_corpus.append(tokens)

print(f"Finished processing {len(processed_corpus)} sentences.")

/Users/wolfpack/uni/NLP/NLP_practical_project/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/wolfpack/uni/NLP/NLP_practical_project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Generating train split: 1128927 examples [00:04, 233913.22 examples/s]
Parallel Tokenizing:   0%|          | 0/247023 [00:00<?, ?it/s]/Users/wolfpack/uni/NLP/NLP_practical_project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnin

Finished processing 247023 sentences.


In [ ]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(
    sentences=processed_corpus,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4
)

w2v_model.train(processed_corpus, total_examples=len(processed_corpus), epochs=10)
w2v_model.save("bible_ro_word2vec.model")

In [12]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tagged_data = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(processed_corpus)]

d2v_model = Doc2Vec(
    vector_size=100,
    window=5,
    min_count=2,
    dm=1,
    epochs=20,
    workers=4
)

d2v_model.build_vocab(tagged_data)
d2v_model.train(tagged_data, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)
d2v_model.save("bible_ro_doc2vec.model")

Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'
Exception ignored in: 'gensim.models.word2vec_inner.our_dot_float'


In [13]:
test_tokens = preprocess("Cuvantul este adevar")
inferred_vector = d2v_model.infer_vector(test_tokens)
sims = d2v_model.dv.most_similar([inferred_vector], topn=10)

for tag, similarity in sims:
    print(original_sentences_list[int(tag)], similarity)

pietre de onix și pietre pentru efod și pieptar. 0.7626572847366333
Fiii lui Reuel au fost:Nahat, Zerah, Șama și Miza. 0.7585799694061279
Fiul lui Nahat a fost Eliab. Fiul lui Eliab a fost Ieroham. Fiul lui Ieroham a fost Elcana. 0.7482082843780518
piei de berbec vopsite în roșu și piei de vițel de mare; lemn de salcâm, 0.744839072227478
Fiii lui Merari au fost:Mahli și Muși.Acestea au fost clanurile lui Levi, potrivit genealogiilor lor. 0.7410770654678345
Fiecare scândură să fie lungă de zece coți și lată de un cot și jumătate. 0.7405774593353271
Însă „Cel ce se laudă să se laude în Domnul“. 0.7392435073852539
Fiii lui Iosíf, după familiile lor, au fost: Manáse și Efraím. 0.7376835346221924
Și foametea era aspră în țară. 0.7363867163658142
Fiii lui Chehat au fost: Amram, Ițhar, Hebron și Uziel. 0.7356805801391602
